In [ ]:
import os
import json

folder_path = "final"
combined_data = []


for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                if isinstance(data, list):  
                    combined_data.extend(data)
                elif isinstance(data, dict):
                    combined_data.append(data)
        except (json.JSONDecodeError, UnicodeDecodeError) as e:
            print(f"Skipping {filename} due to error: {e}")


output_file = "merged.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(combined_data, f, indent=4)

print(f"✅ Merged {len(combined_data)} JSON entries into {output_file}")


: 

In [ ]:
import os
import json
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

folder_path = "final"
combined_data = []


for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                
                if isinstance(data, list):  
                    combined_data.extend(data)
                elif isinstance(data, dict):
                    combined_data.append(data)
        except (json.JSONDecodeError, UnicodeDecodeError) as e:
            print(f"Skipping {filename} due to error: {e}")


for item in combined_data:
    if "materials_required" in item and isinstance(item["materials_required"], list):
        materials_text = ", ".join(item["materials_required"])
        embedding = model.encode(materials_text).tolist() 
        item["embedding"] = embedding 


output_path = "combined_data_with_embeddings.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(combined_data, f, indent=4)

print(f"Processed {len(combined_data)} records. Output saved to {output_path}.")


Processed 748 records. Output saved to combined_data_with_embeddings.json.


In [ ]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


model = SentenceTransformer("all-MiniLM-L6-v2")


with open("combined_data_with_embeddings.json", "r", encoding="utf-8") as f:
    combined_data = json.load(f)


def find_similar_materials(user_materials, top_n=3):

    user_text = ", ".join(user_materials)
    user_embedding = model.encode(user_text).reshape(1, -1)

    similarities = []
    
    for item in combined_data:
        if "embedding" in item:
            stored_embedding = np.array(item["embedding"]).reshape(1, -1)
            similarity_score = cosine_similarity(user_embedding, stored_embedding)[0][0]
            similarities.append((item, similarity_score))


    similarities.sort(key=lambda x: x[1], reverse=True)

 
    return similarities[:top_n]


user_input_materials = ["wood", "nails", "glue"]


top_matches = find_similar_materials(user_input_materials)

# Print results
for i, (match, score) in enumerate(top_matches, 1):
    print(f"\nMatch {i}:")
    print(f"Materials Required: {match['materials_required']}")
    print(f"Steps: {match.get('steps', 'No steps provided')}")
    print(f"Similarity Score: {score:.4f}")



Match 1:
Materials Required: ['Wood', 'Aluminum', 'Uni Joint', 'Bolts', 'Nails', 'Super glue with spray activator', 'Packing tape']
Steps: ['Cut thin strips of wood for bent lamination using a table saw and soak them in water for about an hour to make them flexible.', 'While the wood is soaking, make small wedges on the table saw by tilting the mitre gauge to 10 degrees.', 'Bend the wet strips around a circular form, attaching small pieces of wood to secure the lamination with brad nails, leaving a small gap. Hammer the wedges into the gaps to apply pressure and eliminate gaps, then let it dry overnight.', 'Glue up the bent lamination, applying a lot of glue to all surfaces and covering the form with packing tape to prevent sticking. Use the wedges again to ensure good pressure and no gaps. Wipe off excess glue with a damp cloth.', 'Create the handle using a Uni Joint, drilling a hole near the end of a rod and cutting down the sides. Attach a small piece of wood with two holes at 90 d